In [0]:
%matplotlib inline
!pip install tensorboardcolab
import tensorboardcolab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


 ╔══<i><b>&nbsp;Alai-DeepLearning&nbsp;</b></i>══════════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 5. machine learning basis**
# Homework 2. Multi classification

### _Objective_
Tensorflow 을 이용해서 One Vs All 전략을 이용해 다중 classification 모델을 생성합니다.

아래 순서에 맞게 프로그램을 작성해 주세요.
1. Setosa  Vs Versicolour, Virginica 모델 구현
 - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
2. Versicolour Vs Setosa, Virginica 모델 구현
  - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
3. Virginica Vs Setosa, Versicolour 모델 구현
  - 학습시 Tensorboard 을 이용해 loss 와 accuracy 을 추적해 주세요. 
 - 학습 동안 acc 가 가장 높은 모델을 저장해 주세요.
4. 저장된 3개의 모델을 불러온 후 가장 높은 확률이 나오는 값을 선택합니다. <br>
가령 아래의 경우 **Versicolour** 을 선택합니다.

| class        | probabilty |
|--------------|------------|
| Setosa       | 0.7        |
| Versicolour  | 0.9        |
| Virginica    | 0.3        |

학습이 끝난후 모든 데이터를 평가한 후 accuracy 을 측정합니다.

╚═══════════════════════════════════════════════╝

In [0]:
#np.random.seed(1)    
iris = load_iris()

xs = iris['data']
ys = iris['target']
ys_name = iris['target_names']
ys_name

#standarization
xs = (xs - xs.mean(axis=0))/xs.std(axis=0)
#print(xs.std(axis=0))
#print(xs.shape)

In [0]:
tf.reset_default_graph()
tbc=tensorboardcolab.TensorBoardColab(graph_path='./tensorboard')


Wait for 8 seconds...
TensorBoard link:
https://be894bb5.ngrok.io


In [0]:
class Classifier :
    def __init__(self, one) :
        self.one = one
        self.train_acc_ = 0
        
        #label 만들기
        label = np.array(list(map(lambda ys : 1 if ys != one else 0, ys)))
        label = np.reshape(label, (-1,1))
        #print(label.shape)

        con_xs_ys = np.concatenate((xs,label), axis = 1)

        #train _ test split
        train_df, test_df = train_test_split(con_xs_ys,
                                            stratify=con_xs_ys[:,-1],
                                            train_size=0.8)
        xs_train = train_df[:,:-1]
        #print(xs_train.shape)
        label_train = train_df[:,-1]
        label_train = np.reshape(label_train, (-1,1))

        xs_test = test_df[:,:-1]
        label_test = test_df[:,-1]
        label_test = np.reshape(label_test, (-1,1))

        graph = tf.Graph()
        with graph.as_default() :
            #model 정의
            xs_holder = tf.placeholder(dtype=tf.float32, shape = [None, 4])
            ys_holder = tf.placeholder(dtype=tf.float32, shape = [None,1])
            w = tf.Variable(tf.random.normal(shape=[4,1], mean = 0.0, stddev = 0.1), dtype= tf.float32)
            bias = tf.Variable(tf.random.normal(shape = [1], mean = 0.0, stddev = 0.1), dtype = tf.float32)

            z = tf.add(tf.matmul(xs_holder,w),bias)

            prob = 1/(1 + tf.exp(-z))

            w_del = tf.reduce_mean((prob - ys_holder) * xs_holder, axis = 0)
            w_del_reshape = tf.reshape(w_del, shape = (-1,1))
            w_new = w - (0.01 * w_del_reshape)
            asig_w = tf.assign(w, w_new)

            b_del = tf.reduce_mean((prob - ys_holder), axis=0)
            #b_del_reshape = tf.reshape(b_del, shape = (-1))
            b_new = bias - (0.01 * b_del)
            asig_b = tf.assign(bias, b_new)

            train = tf.group([asig_w, asig_b])

            cross_entro = -(tf.reduce_mean(ys_holder*tf.log(prob) + (1-ys_holder)*tf.log(1-prob)))
            tf.summary.scalar(name = 'loss_sc' , tensor = cross_entro)

            cut_val = tf.constant(0.5, dtype=tf.float32)
            g_e = tf.greater_equal(prob, cut_val)
            one = tf.ones(tf.shape(g_e),dtype = tf.float32)
            zeros = tf.zeros(tf.shape(g_e), dtype = tf.float32)
            cut_res = tf.where(g_e, one, zeros)
            equ = tf.where(tf.equal(cut_res, ys_holder), one, zeros)
            #print(equ.get_shape())
            acc = tf.reduce_mean(equ)
            #print(tf.shape(acc))
            tf.summary.scalar(name = 'acc_sc', tensor = acc)

            merge = tf.summary.merge_all()
            with tf.Session() as sess :
                sess.run(tf.global_variables_initializer())
                writer = tf.summary.FileWriter(logdir='./tensorboard', graph = sess.graph)

                for i in range(100) :
                    _, merge_, loss_, acc_ = sess.run([train, merge, cross_entro, acc],
                                                      feed_dict = {xs_holder : xs_train,
                                                                   ys_holder : label_train})

                    #acc_test = sess.run([acc], feed_dict = {xs_holder : xs_test, ys_holder : label_test})

                    writer.add_summary(merge_, i)
                    #print("step : {}".format(i))
                    #print("train_acc : {}".format(acc_))
                    #print("train_loss : {}".format(loss_))
                    #print("test_acc : {}".format(acc_test))
                    #print("-------------------")
                saver = tf.train.Saver()
                saver.save(sess, './tmp/model{}'.format(self.one))    
                self.train_acc_ = acc_
        
    def print_acc(self):
        print(self.train_acc_)
        return self.train_acc_
        

In [0]:
list_res = []
c1 = Classifier(0)
res = c1.print_acc()
list_res.append(res)

c2 = Classifier(1)
res = c2.print_acc()
list_res.append(res)

c3 = Classifier(2)
res = c3.print_acc()
list_res.append(res)

!ls ./tmp/

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


1.0
0.65833336


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.85833335
[1.0, 0.65833336, 0.85833335]
checkpoint		    model1.meta
model			    model2.data-00000-of-00001
model0.data-00000-of-00001  model2.index
model0.index		    model2.meta
model0.meta		    model.data-00000-of-00001
model1.data-00000-of-00001  model.index
model1.index		    model.meta


In [95]:
idx_max = np.argsort(list_res)[-1] #제일큰 acc index

print('가장 높은확률로',ys_name[idx_max],"입니다.")

# gra = tf.Graph()
# with gra.as_default() :
#     idx_max = np.argsort(list_res)[-1] #제일큰 acc index

#     #label 만들기
#     label = np.array(list(map(lambda ys : 1 if ys != one else 0, ys)))
#     label = np.reshape(label, (-1,1))
#     #print(label.shape)

#     con_xs_ys = np.concatenate((xs,label), axis = 1)

#     #train _ test split
#     train_df, test_df = train_test_split(con_xs_ys,
#                                         stratify=con_xs_ys[:,-1],
#                                         train_size=0.8)
#     xs_train = train_df[:,:-1]
#     #print(xs_train.shape)
#     label_train = train_df[:,-1]
#     label_train = np.reshape(label_train, (-1,1))

#     xs_test = test_df[:,:-1]
#     label_test = test_df[:,-1]
#     label_test = np.reshape(label_test, (-1,1))


#     with tf.Session() as sess :
#         new_saver = tf.train.import_meta_graph('./tmp/model{}.meta'.format(idx_max))
#         new_saver.restore(sess,'/tmp')
#         acc = graph.get_tensor_by_name('acc:0')
#         acc_ = sess.run([acc],feed_dict = {xs_holder : xs_train,
#                                            ys_holder : label_train})

#         print("acc:", acc_)

가장 높은확률로 setosa 입니다.


저장된 변수의 이름을 불러옵니다
저장된 변수의 이름 [('Variable', [4, 1]), ('Variable_1', [1])]
w : [[-0.18142082]
 [ 0.01615225]
 [-0.20999752]
 [-0.3797823 ]] w1 : [0.0137773]


#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/03/22

---